# Test WaffleIron Model

In [1]:
from models.waffleiron.segmenter import Segmenter

Using torch.scatter_reduce for 3D to 2D projection.
Using torch.scatter_reduce for 3D to 2D projection.


In [2]:
model = Segmenter(
    input_channels=5,
    feat_channels=768,
    depth=48,
    grid_shape=[[256, 256], [256, 32], [256, 32]],
    nb_class=16,
    #drop_path_prob=config["waffleiron"]["drop_path"],
    layer_norm=True,
)

In [4]:
import torch

# Load pretrained model
ckpt = torch.load('./saved_models/ckpt_last_scalr.pth', map_location="cpu")
ckpt = ckpt["net"]

In [5]:
print(ckpt.keys())

dict_keys(['module.embed.norm.weight', 'module.embed.norm.bias', 'module.embed.norm.running_mean', 'module.embed.norm.running_var', 'module.embed.norm.num_batches_tracked', 'module.embed.conv1.weight', 'module.embed.conv1.bias', 'module.embed.conv2.0.weight', 'module.embed.conv2.0.bias', 'module.embed.conv2.0.running_mean', 'module.embed.conv2.0.running_var', 'module.embed.conv2.0.num_batches_tracked', 'module.embed.conv2.1.weight', 'module.embed.conv2.2.weight', 'module.embed.conv2.2.bias', 'module.embed.conv2.2.running_mean', 'module.embed.conv2.2.running_var', 'module.embed.conv2.2.num_batches_tracked', 'module.embed.conv2.4.weight', 'module.embed.final.weight', 'module.embed.final.bias', 'module.waffleiron.channel_mix.0.scale.weight', 'module.waffleiron.channel_mix.0.norm.weight', 'module.waffleiron.channel_mix.0.norm.bias', 'module.waffleiron.channel_mix.0.mlp.0.weight', 'module.waffleiron.channel_mix.0.mlp.0.bias', 'module.waffleiron.channel_mix.0.mlp.2.weight', 'module.waffleiro

In [6]:
new_ckpt = {}
for k in ckpt.keys():
    if k.startswith("module"):
        if k.startswith("module.classif.0"):
            continue
        elif k.startswith("module.classif.1"):
            new_ckpt["classif" + k[len("module.classif.1") :]] = ckpt[k]
        else:
            new_ckpt[k[len("module.") :]] = ckpt[k]
    else:
        new_ckpt[k] = ckpt[k]

In [7]:
new_ckpt.get("classif.weight").shape

torch.Size([16, 768, 1])

In [8]:
model.load_state_dict(new_ckpt)

<All keys matched successfully>

## Model loaded --> Test features somehow

In [13]:
import os
import torch
from auxiliary.process_data.nuscenes.nuscenes_dataset import DatasetTrainVal

print("Creating dataloader...", flush=True)

target = '/root/main/dataset/processed'

filelist_train = [os.path.join(target, 'train_pointclouds', fname) for fname in os.listdir(os.path.join(target, 'train_pointclouds')) if os.path.splitext(fname)[1]==".npy"]
filelist_train.sort()
filelist_val = filelist_train[:3]
filelist_train = filelist_train[3:]

ds = DatasetTrainVal(filelist_train, os.path.join(target, 'train_pointclouds'),
                            training=True,
                            npoints=2000,
                            iteration_number=(2*10)*10,
                            jitter=0.2)
train_loader = torch.utils.data.DataLoader(ds, batch_size=1, shuffle=False, # Change batch_size
                                    num_workers=1)

Creating dataloader...
Len: 82


In [18]:
for t in train_loader:
    print(torch.bincount(t['target'][0]))
    x = input()

tensor([1244,  274,    0,    0,    0,    0,    0,  482])

tensor([1191,  376,    0,    0,    0,    0,    0,  433])

tensor([1149,  277,    0,    0,    0,    0,    0,  574])

tensor([1139,  272,    0,    0,    0,    0,    0,  589])

tensor([1059,  343,    0,    0,    0,    0,    0,  598])

tensor([1264,  448,    0,    0,    0,    0,    0,  288])

tensor([1208,   68,    0,    0,    0,    0,    0,  724])

tensor([1421,  133,    0,    0,    0,    0,    0,  446])

tensor([1095,  317,    0,    0,    0,    0,    0,  588])

tensor([1157,  145,    0,    0,    0,    0,    0,  698])

tensor([1089,  320,    0,    0,    0,    0,    0,  591])

tensor([1084,  284,    0,    0,    0,    0,    0,  632])

tensor([1059,  360,    0,    0,    0,    0,    0,  581])

tensor([1114,  247,    0,    0,    0,    0,    0,  639])

tensor([1138,  285,    0,    0,    0,    0,    0,  577])

tensor([994, 552,   0,   0,   0,   0,   0, 454])


KeyboardInterrupt: Interrupted by user

In [ ]:
from plyfile import PlyData, PlyElement

train_filenames = ["Lille1_1.ply",  "Lille1_2.ply",  "Lille2.ply",  "Paris.ply",]
destdir = '/root/main/dataset/processed/'
filenames = train_filenames
save_dir = os.path.join(destdir,"train_pointclouds")
for filename in filenames:
    fname = os.path.join(args.rootdir, "training_10_classes", filename)
    plydata = PlyData.read(fname)
    x = plydata["vertex"].data["x"].astype(np.float32)
    y = plydata["vertex"].data["y"].astype(np.float32)
    z = plydata["vertex"].data["z"].astype(np.float32)
    reflectance = plydata["vertex"].data["reflectance"].astype(np.float32)
    label = plydata["vertex"].data["class"].astype(np.float32)
    print(torch.bincount(label))